In [1]:
from nltk import word_tokenize
from gensim import corpora
import pandas as pd
import numpy as np
import re
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
cs_path = '/home/grad24/dangpnh/dangpnh/HSLN-Joint-Sentence-Classification/data/PubMed_20k_RCT/'

In [2]:
def read_text_format(file_name):
    meta_ = []
    sents_ = []
    labels_ = []
    with open(file_name, errors='ignore') as f:

        doc_sent_count = 0
        for idx, line in enumerate(f):
            # start doc
            if "###" in line:
                if doc_sent_count!=0:
                    meta_.append(doc_sent_count)
                doc_sent_count = 0
                continue
            if '\t' in line:
                label, sent = line.split('\t')
                sents_.append(sent)
                labels_.append(label)
                doc_sent_count+=1
    meta_.append(doc_sent_count)
    return meta_, sents_, labels_

In [3]:
## Load intent label for sentence level
train_file = cs_path + "train_clean.txt"
dev_file = cs_path + "dev_clean.txt"
test_file = cs_path + "test_clean.txt"


meta_train, sents_train, labels_train = read_text_format(train_file)
meta_dev, sents_dev, labels_dev = read_text_format(dev_file)
meta_test, sents_test, labels_test = read_text_format(test_file)

In [4]:
meta_ = meta_train+meta_dev+meta_test
sents_ = sents_train+sents_dev+sents_test
labels_ = labels_train+labels_dev+labels_test

In [5]:
puncs = '!"#$%&\'()*+,-/:;<=>@[\]^_`{|}~'
stop_words_analogy = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", 
                      "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", 
                      "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", 
                      "their", "theirs", "themselves", "what", "which", "who", "whom", "this", 
                      "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", 
                      "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", 
                      "an", "the", "and", "if", "or", "as", "of", "at", "by", "for", "with", "about", 
                      "into", "through", "to", "from", "up", "down", "in", "out", "on", "off", "over",
                      "under", "then", "once", "here", "there", "when", "where", "why", "how", "all", 
                      "any", "both", "each", "other", "some", "such", "no", "nor", "not", "own", "same", 
                      "so", "than", "too", "very", "can", "just", "don", "should"]
# Lowercase, trim, and remove non-letter characters
def normalizeString(s):

    s = s.lower().strip()
    for c in list(puncs):
        s=s.replace(c, " ")
    s = word_tokenize(s)
    s = filter(lambda x:x not in stop_words_analogy, s)
    s = ' '.join(e for e in s)
    s = re.sub("(?<!\S)\d+(?!\S)", "", s) ## REMOVE NUMBERS 

    return s
sents_ = [normalizeString(e) for e in sents_]

In [6]:
doc_list_split = [doc.split() for doc in sents_]
dictionary = corpora.Dictionary(doc_list_split)
dictionary.filter_extremes(no_below=0, no_above=0.9,keep_n=5000)
corpus = np.array([dictionary.doc2bow(line) for line in doc_list_split])

/home/grad24/dangpnh/miniconda3/envs/py36_latest_torch/lib/python3.6/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


In [7]:
list_words = []
for k, v in dictionary.token2id.items():
    list_words.append(k)
list_words_idx = {k: v for v, k in enumerate(list_words)}
def keep_w(sent):
    new_l = []
    words = sent.split()
    for w in words:
        if w in list_words:
            new_l.append(w)
    return ' '.join(e for e in new_l)

In [8]:
def group_list(l, group_size):
    """
    :param l:           list
    :param group_size:  size of each group
    :return:            Yields successive group-sized lists from l.
    """
    for i in xrange(0, len(l), group_size):
        yield l[i:i+group_size]

In [9]:
# sents_ = [keep_w(e) for e in sents_]

In [10]:
def split_doc_len(doc_len, max_len):
    assert doc_len > max_len
    last_numb = []
    if doc_len%max_len:
        last_numb = [doc_len%max_len]
    return [max_len]*int(doc_len/max_len)#+last_numb
    
    

In [11]:
max(meta_)

31

In [12]:
MAX_SENTS = max(meta_)

In [13]:
new_sents_ = []
new_labels_ = []
new_meta_ = []

curr_sent = 0

for doc_len in meta_:
    
    if doc_len > MAX_SENTS:
        splitted_doc_len = split_doc_len(doc_len, MAX_SENTS)
    else:
        splitted_doc_len = [doc_len]
    
    for sub_doc_len in splitted_doc_len:
        total_sent = 0
        for idx in range(curr_sent, curr_sent+sub_doc_len):
            sent_words = keep_w(sents_[idx])
            if len(sent_words)>3:

                new_sents_.append(sent_words)
                new_labels_.append(labels_[idx])
                total_sent+=1

        curr_sent+=sub_doc_len
        if total_sent>0:
            new_meta_.append(total_sent)
    
assert sum(new_meta_) == len(new_sents_)

labels_ = new_labels_
meta_ = new_meta_
sents_ = new_sents_

In [14]:
keys = list(set(labels_))
values = list(range(1,1+len(keys)))
dict_ = dict(zip(keys, values))

int_labels = []
for l in labels_:
    int_labels.append(dict_[l])

In [15]:
dict_

{'OBJECTIVE': 1, 'RESULTS': 2, 'BACKGROUND': 3, 'CONCLUSIONS': 4, 'METHODS': 5}

In [16]:
inv_map = {v-1: k for k, v in dict_.items()}

In [17]:
inv_map

{0: 'OBJECTIVE', 1: 'RESULTS', 2: 'BACKGROUND', 3: 'CONCLUSIONS', 4: 'METHODS'}

In [18]:
save_path = '/home/grad24/dangpnh/dangpnh/analogy_vae/data/pubmed/'
label_file = save_path+'pubmed.label'
labelstr_file = save_path+'pubmed.strlabel'
meta_file = save_path+ 'pubmed.meta'
text_file = save_path+'pubmed.text'

# write labels in integer
with open(label_file, 'w') as f:
    for line in int_labels:
        f.write(f"{line}\n")

# write labels in str
with open(labelstr_file, 'w') as f:
    for line in labels_:
        f.write(f"{line}\n")

# write meta
with open(meta_file, 'w') as f:
    for line in meta_:
        f.write(f"{line}\n")

# write text
with open(text_file, 'w') as f:
    for line in sents_:
        line=line.strip()
        f.write(f"{line}\n")

In [8]:
## Write vocab, .data

In [9]:

save_path = '/home/grad24/dangpnh/dangpnh/analogy_vae/data/pubmed20k/'
list_words_dict = {k: v for k,v in enumerate(list_words)}
inv_list_words_dict = {v: k for k, v in list_words_dict.items()}

In [10]:
# ## write vocab; .data; vocab file start from 1; data start from 0
with open(save_path+"pubmed20k.vocab", "w") as f:
    for k,v in list_words_dict.items():
        f.write(f"{k+1},{v}\n")

In [11]:

with open(save_path+"pubmed20k.text", "r") as f:
    original_text = f.readlines()


In [16]:
from collections import Counter
new_sents = []
for sent in original_text:
    words = sent.split()
    c_words = dict(Counter(words))
    data_sent = ','.join([str(inv_list_words_dict[k])+":"+str(v) for k,v in c_words.items() if k in inv_list_words_dict])
    new_sents.append(data_sent)

In [17]:
with open(save_path+"pubmed20k.data", "w") as f:
    for i in new_sents:
        f.write(f"{i}\n")